## Performance improvements

* Invoked sparsity throughout analysis: significant speedup
* Re-wrote QCQP matrix generation methods, 90% reduction in memory allocation
* Wrapped core computation in parallelizable for loop
* Plotted computation time vs memory allocation for various numbers of worker processes *(plot below)*

![Parallel performance](../images/2015-07-29-parallelization-performance.svg)

* Re-wrote `find_x_star` to handle cases where $A$ doesn't have full row rank
* Added zero-resistance check: if a line has zero resistance, no amount of power flow can influence its temperature, so skip it.
* Created type containers for input data and output data; makes it easier to load data for any network or plot any analysis output regardless of conductor parameters.
* ** 6-time-step RTS-96 analysis time: **

| Time (s) | Note                        |
|:--------:|:----------------------------|
| 420      | Original                    |
| 20       | No more `backslash`         |
| 13       | Invoked sparsity            |
| 8        | Fine-tuning                 |
| 4        | Parallelize to 2 processors |

## Testing on multiple networks

* Wrote code to supplement data for any Matpower network with RTS-96 thermal data
* Performed instanton analysis on nine Matpower networks (with RTS-96-esque thermal data) to characterize performance *(plot below)* 

![Performance by network](../images/2015-08-11-performance-by-network.svg)

## Covariance matrices

* Pass [randomly-generated][1] unit-norm covariance matrix to instanton analysis code.
* Observed no discernible performance implications.
* Observed downward shift in instanton candidate scores for 6-time-step RTS-96 analysis *(plot below)*

![Score by covariance](../images/2015-08-11-score-by-covariance.svg)

[1]: http://dx.doi.org/10.1016/j.jmva.2009.04.008

## Roadmap

* Continue writing about findings, refining existing draft
* Better visualization of covariance effects
* Finer control of covariance matrix generation
* Study temporal covariance
* Attempt to use `eigs` rather than `eig` to speed up computation on larger networks